In [ ]:
import numpy as np #compulsory meth
import matplotlib.pyplot as plt #graphing
import pandas as pendar #pendar
import sklearn as sk #regression
import chempy as che #chemistry
import scipy as sci #science calculations
import pyomo as pyo #optimization
import chemicals as cls #chemical properties
import ht #heat transfer library


# Python HRS - https://www.youtube.com/watch?v=3g301ybQKos

#inputs
#ambient (P,T), H2 (P,T), energy (kW), Water (kg/s, P, T), H2 (flowrate), controller variables

#outputs
#H2 (production rate), oxygen (production rate), fluid (P,T)

In [ ]:
#def PEM (Gordon, Vincent)
#Chemical EQN: H2O -> 1/2 O2 + H2
#Nernst EQN, Faraday's Law, Efficiency
def calculate_output_PEM(I_in, V_in, H2O_in, Energy_in, H2O_T, H2O_P, eff, R=8.314):
    dH2_dT = eff*I_in/(2*F_const) #Hydrogen production mol/s
    dO2_dT = dH2_dT/2 #Oxygen production mol/s
    dH2O_dT = H2O_in - dH2_dT #Water production mol/s

    #cell temperature (vary between -20C to 40C)
    Q_net = (I_in**2)*Res_e*t_op - 285830*dH2O_dT #electrolyte resistance heat - heat of reaction (endo)
    del_T = Q_net/(H2O_spec_heat*H2O_in*t_op) #per day
    Cell_T = Cell_T + del_T #per day

    #cell voltage
    #E_cell = 1.23-0.0009*(Cell_T-298)+(R*Cell_T/(4*F_const))*np.log((Pv_O2*(Pv_H2)**2)/Pv_H2O) #https://core.ac.uk/download/pdf/52394289.pdf
    I_den = I_in/Area
    V_cell = 1.23-0.18545*np.log(I_den)-0.10935*(I_den)-(-3.67397)*np.exp(-1.45761e-06)-0.18545*np.log(P/P_O2) #empirical formula from lab A1 Dylan -> can use non ideal gas to find partial P of O2
    Pwr_Req = V_cell * I_in #Power requirement

    return dH2_dT,dO2_dT,dH2O_dT,Cell_T,Pwr_Req

MW_H2 = 1.00794 #Molecular Weight of H2
MW_O2 = 31.9988 #Molecular Weight of O2
MW_H2O = 18.01528 #Molecular Weight of H2O
F_const = 96485.31 #faraday's constant
Cell_T = 293.15 #ambient cell temperature at start
t_op = 86400 #operation time in seconds of a day
Res_e = 1 #resistance of electrolyte
H2O_spec_heat = 4.184 #kj/kgK
PEM_eff = input("What is the efficiency %?: ") #75% according to H-TEC systems
OutP_H2 = 25 #Hydrogen output pressure (bar)


What is the efficiency %?75


In [ ]:
#def compressor to 200 bar (Rayan, Anand)
#RK equation https://chemicalprojects.wordpress.com/2014/07/06/equations-of-state/
#assumed 50°C temp for hydrogen
def calculate_rk_constants(Tc, Pc, R=8.314):
    """
    Calculate Redlich-Kwong constants 'a' and 'b' from critical temperature and pressure.

    Parameters:
    - Tc: Critical temperature in Kelvin
    - Pc: Critical pressure in Pascals
    - R: Gas constant in J/(mol K)

    Returns:
    - a: Redlich-Kwong constant 'a' in (Pa m^6/mol^2)
    - b: Redlich-Kwong constant 'b' in (m^3/mol)
    """
    a = 0.427 * ((R**2 * Tc_hydrogen**2.5) / Pc_hydrogen)
    b = 0.08664 * (R * Tc_hydrogen / Pc_hydrogen)
    return a, b
def calculate_compressibility_factor(P, V, T, R=8.314):
    """
    Calculate compressibility factor using the Redlich-Kwong equation.

    Parameters:
    - P: Pressure in Pascals
    - V: Molar volume in m^3/mol
    - T: Temperature in Kelvin
    - R: Gas constant in J/(mol K)

    Returns:
    - Z: Compressibility factor
    """
    virialB = 14.87        #assumed 50°C temp for hydrogen https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5325166/
    virialC = 340          #assumed 50°C temp for hydrogen
    Bprime = virialB/(R*T)
    Cprime = (virialC-Bprime**2)/(R*T)**2
    #Z = 1 + Bprime*P + P**2*Cprime
    Z = (P*V_hydrogen)/(R*T_hydrogen)
    return Z


def compressour_output(T1, P1, P2_output, R_H2_gas, V_H2,eta_isentropic,Tc_hydrogen, Pc_hydrogen):
  # isentropic compression
  # Calculate RK constants for hydrogen
  a_hydrogen, b_hydrogen = calculate_rk_constants(Tc_hydrogen, Pc_hydrogen)

  # Calculate compressibility factor
  Z_inlet = calculate_compressibility_factor(P1, V_H2, T1)

  # isentropic outlet conditions
  V2_isentropic = 1 / Z_inlet * (P2_output / P1)

  # Actual volume using isentropic efficiency
  V2_actual = V2_isentropic + eta_isentropic * (V2_isentropic - 1)

  # Actualoutlet temperature and pressure
  T2_actual = T1 * (V2_isentropic / V2_actual)
  P2_actual = R_H2_gas * T2_actual / V2_actual - a_hydrogen / (V2_actual * (R_H2_gas * T2_actual) ** 0.5)

  return T2_actual, P2_actual

In [ ]:
# Given conditions
T_hydrogen = 273.15 + 45  # Inlet to compressor (Temperature in Kelvin)
Tc_hydrogen = 33            # Critical temperature in Kelvin
Pc_hydrogen = 189*6895      # Critical Pressure in Pa
R_hydrogen = 4124  # Gas constant for hydrogen, J/(mol K)
eta_isentropic_assumed = 0.8  # Assume isentropic efficiency: can be 0.7 to 0.8
P_hydrogen = 25*100000     # Inlet Pressure in Pa (25 bar)
P_hydrogenatm = 25/1.013     # Inlet Pressure in atm
R = 0.08206 # L * atm / mol * K
V_hydrogen = ((T_hydrogen*R)/(P_hydrogenatm))/1000     # Molar volume in m^3/mol

# Calculate RK constants for hydrogen
a_hydrogen, b_hydrogen = calculate_rk_constants(Tc_hydrogen, Pc_hydrogen)

# Calculate compressibility factor
Z_hydrogen = calculate_compressibility_factor(P_hydrogen, V_hydrogen, T_hydrogen)

# compressor outputs
#T_output_comp1, P_output_comp1 = compressour_output(T_hydrogen, OutP_H2*100000, 200*100000, R_hydrogen, V_H2, eta_isentropic_assumed,Tc_hydrogen, Pc_hydrogen) # what would be desired outlet hydrogen pressure? 200 bar?

print(f"Compressibility Factor for Hydrogen (Z): {Z_hydrogen}")
print(f"Redlich-Kwong constants for Hydrogen - a: {a_hydrogen}, b: {b_hydrogen}")
#print(f"Compressor outlet conditions - T: {T_output_comp1}, P: {P_output_comp1}")

Compressibility Factor for Hydrogen (Z): 0.9998409910993504
Redlich-Kwong constants for Hydrogen - a: 0.14168918113229173, b: 1.824090279360475e-05


In [ ]:
#compressor to transport ratio split

In [ ]:
#Transportation

In [ ]:
#Cascading

In [ ]:
#Heat Exchanger

In [ ]:
#Nozzle

In [ ]:
#Tilbury South

In [ ]:
#Tilbury North

In [ ]:
#Ingersoll W

In [ ]:
#Woodstock E

In [ ]:
#Cambridge South

In [ ]:
#Cambridge North

In [ ]:
#Newcastle W

In [ ]:
#Port Hope E

In [ ]:
#Mallory Town South

In [ ]:
#Mallory Town North

In [ ]:
#Ingleside E